In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv('Groceries_dataset.csv', parse_dates=[1]) 

In [3]:
df.dtypes

Member_number               int64
Date               datetime64[ns]
itemDescription            object
dtype: object

In [4]:
df.isna().sum()

Member_number      0
Date               0
itemDescription    0
dtype: int64

In [5]:
df

,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-05-01,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-01-02,whole milk
...,...,...,...
38760,4471,2014-08-10,sliced cheese
38761,2022,2014-02-23,candy
38762,1097,2014-04-16,cake bar
38763,1510,2014-03-12,fruit/vegetable juice


The format of the data is the id, data and the item bought (i.e. the item description column). 
Since the itemDescription column has only 1 item, we first group it by the id and the date, to create itemsets (also known as baskets).

The issue with this grouping is that we cannot be sure that the items are bought during the same visit, or doing multiple visits in the same day. However, since there is no other way, we assume that no one visited the store more than once a day

Since the dataset is in one column, we need to group it based on date and member number
We will have to make the assumption that one member does not visit the store twice in one day, since we only have date of visit, not the time.

In [6]:
# First, get a list of all items. Easiest way to do this is by dummyfying the item description column
items_dummies = pd.get_dummies(df['itemDescription'])

In [7]:
# Baskets will be a list of items bought together
baskets = df.groupby(['Date', 'Member_number']).agg(lambda x: ','.join(x).split(','))['itemDescription'].values

In [8]:
baskets, len(baskets)

(array([list(['citrus fruit', 'coffee']), list(['curd', 'soda']),
        list(['other vegetables', 'yogurt']), ...,
        list(['domestic eggs', 'cream cheese ']),
        list(['canned beer', 'canned beer']), list(['dessert', 'curd'])],
       dtype=object),
 14963)

In [9]:
df = df.join(items_dummies)

In [10]:
df

,Member_number,Date,itemDescription,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,2015-07-21,tropical fruit,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2552,2015-05-01,whole milk,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2300,2015-09-19,pip fruit,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1187,2015-12-12,other vegetables,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,3037,2015-01-02,whole milk,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38760,4471,2014-08-10,sliced cheese,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38761,2022,2014-02-23,candy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38762,1097,2014-04-16,cake bar,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38763,1510,2014-03-12,fruit/vegetable juice,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.drop('itemDescription', axis=1, inplace=True)

In [12]:
df = df.groupby(['Date', 'Member_number']).sum() 

In [13]:
df.head()

Instant food products  UHT-milk  abrasive cleaner  \
Date       Member_number                                                      
2014-01-01 1249                               0         0                 0   
           1381                               0         0                 0   
           1440                               0         0                 0   
           1659                               0         0                 0   
           1789                               0         0                 0   

                          artif. sweetener  baby cosmetics  bags  \
Date       Member_number                                           
2014-01-01 1249                          0               0     0   
           1381                          0               0     0   
           1440                          0               0     0   
           1659                          0               0     0   
           1789                          0               0     0   

                          baking powder  bathroom cleaner  beef  berries  ...  \
Date       Member_number                                                  ...   
2014-01-01 1249                       0                 0     0        0  ...   
           1381                       0                 0     0        0  ...   
           1440                       0                 0     0        0  ...   
           1659                       0                 0     0        0  ...   
           1789                       0                 0     0        0  ...   

                          turkey  vinegar  waffles  whipped/sour cream  \
Date       Member_number                                                 
2014-01-01 1249                0        0        0                   0   
           1381                0        0        0                   0   
           1440                0        0        0                   0   
           1659                0        0        0                   0   
           1789                0        0        0                   0   

                          whisky  white bread  white wine  whole milk  yogurt  \
Date       Member_number                                                        
2014-01-01 1249                0            0           0           0       0   
           1381                0            0           0           0       0   
           1440                0            0           0           0       1   
           1659                0            0           0           0       0   
           1789                0            0           0           0       0   

                          zwieback  
Date       Member_number            
2014-01-01 1249                  0  
           1381                  0  
           1440                  0  
           1659                  0  
           1789                  0  

[5 rows x 167 columns]

In [14]:
df['basket'] = baskets
df.head(1) # Added the basket column to the dataset

,,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback,basket
Date,Member_number,,,,,,,,,,,,,,,,,,,,,
2014-01-01,1249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[citrus fruit, coffee]"


In [15]:
# Check if the calculation is ok
(df.sum(axis=1) != df['basket'].apply(len)).sum() # Perfect

0

In [16]:
# There are samples where there are more than one of the same item in the basket (eg. {milk, milk})
# We need to only keep 1
len(np.where(df.drop('basket', axis=1)>1)[0])

732

In [17]:
for i in df.drop('basket', axis=1):
    df[i] = df[i].map(lambda x: 1 if x >1 else x)

In [18]:
df

Instant food products  UHT-milk  abrasive cleaner  \
Date       Member_number                                                      
2014-01-01 1249                               0         0                 0   
           1381                               0         0                 0   
           1440                               0         0                 0   
           1659                               0         0                 0   
           1789                               0         0                 0   
...                                         ...       ...               ...   
2015-12-30 3738                               0         0                 0   
           3971                               0         0                 0   
           4058                               0         0                 0   
           4565                               0         0                 0   
           4863                               0         0                 0   

                          artif. sweetener  baby cosmetics  bags  \
Date       Member_number                                           
2014-01-01 1249                          0               0     0   
           1381                          0               0     0   
           1440                          0               0     0   
           1659                          0               0     0   
           1789                          0               0     0   
...                                    ...             ...   ...   
2015-12-30 3738                          0               0     0   
           3971                          0               0     0   
           4058                          0               0     0   
           4565                          0               0     0   
           4863                          0               0     0   

                          baking powder  bathroom cleaner  beef  berries  ...  \
Date       Member_number                                                  ...   
2014-01-01 1249                       0                 0     0        0  ...   
           1381                       0                 0     0        0  ...   
           1440                       0                 0     0        0  ...   
           1659                       0                 0     0        0  ...   
           1789                       0                 0     0        0  ...   
...                                 ...               ...   ...      ...  ...   
2015-12-30 3738                       0                 0     0        0  ...   
           3971                       0                 0     0        0  ...   
           4058                       0                 0     0        0  ...   
           4565                       0                 0     0        0  ...   
           4863                       0                 0     0        0  ...   

                          vinegar  waffles  whipped/sour cream  whisky  \
Date       Member_number                                                 
2014-01-01 1249                 0        0                   0       0   
           1381                 0        0                   0       0   
           1440                 0        0                   0       0   
           1659                 0        0                   0       0   
           1789                 0        0                   0       0   
...                           ...      ...                 ...     ...   
2015-12-30 3738                 0        0                   0       0   
           3971                 0        0                   0       0   
           4058                 0        0                   0       0   
           4565                 0        0                   0       0   
           4863                 0        0                   0       0   

                          white bread  white wine  whole milk  yogurt  \
Date       Member_numbe

In [19]:
len(np.where(df.drop('basket', axis=1)>1)[0])

0

In [20]:
df['UHT-milk'].sum()/len(df) # An example of calculating support for an item

0.021386085678005748

In [21]:
# Drop the item list from the dataframe. No longer needed, since we have verified that the encoding is correct.
df.drop('basket', axis=1, inplace=True)

The support is the probability of an item (or itemset) being bought. Companies usually ignore promoting or working on products that have low support, since there is no point promoting a product that in any case hardly sells. 

In [22]:
# Lets keep the support threshold at 0.1%
supports = apriori(df, min_support=1e-3, use_colnames=True)

In [23]:
# Down to 69 items. How many associations can we have?
supports

,support,itemsets
0,0.004010,(Instant food products)
1,0.021386,(UHT-milk)
2,0.001470,(abrasive cleaner)
3,0.001938,(artif. sweetener)
4,0.008087,(baking powder)
...,...,...
745,0.001136,"(rolls/buns, whole milk, sausage)"
746,0.001002,"(rolls/buns, whole milk, soda)"
747,0.001337,"(rolls/buns, whole milk, yogurt)"
748,0.001069,"(whole milk, sausage, soda)"


In [24]:
# Number of itemsets which have over 1 item in the basket
supports[supports['itemsets'].map(lambda x: len(x)>1)] 

,support,itemsets
149,0.001069,"(UHT-milk, bottled water)"
150,0.002139,"(UHT-milk, other vegetables)"
151,0.001804,"(UHT-milk, rolls/buns)"
152,0.001002,"(UHT-milk, root vegetables)"
153,0.001136,"(UHT-milk, sausage)"
...,...,...
745,0.001136,"(rolls/buns, whole milk, sausage)"
746,0.001002,"(rolls/buns, whole milk, soda)"
747,0.001337,"(rolls/buns, whole milk, yogurt)"
748,0.001069,"(whole milk, sausage, soda)"


It is good to have a threshold metric for association rules.
The two main options are "confidence" and "lift".

Confidence is the proportion of all baskets of the selected antecendent itemset that also contains the consequent item. For eg, if 70% of all baskets with {Egg, Cheese} also contains {Milk}, then the confidence in the rule is 70%. 

Lift is the influence the antecedent itemset has on the consequent item. For instance, if 70% of baskets with {Egg, Cheese} contain {Milk}, but overall, 80% of all itemsets contain milk, then the lift is 70%/80% = 0.875. This means that although the confidence in the rule is high, the probability of the customer buying milk if he/she has egg and cheese in the basket actually decreases.

Using confidence as a pruning metric may work in certain scenarios, but in general, it is always better to use lift.
The minimum threshold should be over 1. However, since the data we have is extremely limited, I have gone with a threshold of 1.

In [25]:
associations = association_rules(supports, metric='lift', min_threshold=1)

In [26]:
# Antecendent support is support for the antecedent, i.e. before adding the new item
# Consequent support is the support of the consequent, i.e. the new item (in the row 127, the consequent support
# is the same as the support of sausage)
associations = associations.sort_values('confidence', ascending=False)


The table below is sorted by confidence. The topmost association {Yogurt, Sausage} -> Whole Milk has a confidence of 0.25, i.e. 25% of all baskets containing yogurt and sausage also contains whole milk.

Similarly, 21% of items containing {Rolls/Buns, Sausage} also contains Whole Milk. 

Whole milk is highly prevalent in this table, since it's individual support is 15.7%, i.e. 15.7% of all people who buy from the store buy milk.

In [27]:
associations.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
236,"(yogurt, sausage)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,0.000563,1.131541
217,"(rolls/buns, sausage)",(whole milk),0.005347,0.157923,0.001136,0.212500,1.345594,0.000292,1.069304
230,"(soda, sausage)",(whole milk),0.005948,0.157923,0.001069,0.179775,1.138374,0.000130,1.026642
202,(semi-finished bread),(whole milk),0.009490,0.157923,0.001671,0.176056,1.114825,0.000172,1.022008
223,"(rolls/buns, yogurt)",(whole milk),0.007819,0.157923,0.001337,0.170940,1.082428,0.000102,1.015701
234,"(whole milk, sausage)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,0.000701,1.093681
112,(detergent),(whole milk),0.008621,0.157923,0.001403,0.162791,1.030824,0.000042,1.005814
147,(ham),(whole milk),0.017109,0.157923,0.002740,0.160156,1.014142,0.000038,1.002659
181,(processed cheese),(rolls/buns),0.010158,0.110005,0.001470,0.144737,1.315734,0.000353,1.040610
177,(packaged fruit/vegetables),(rolls/buns),0.008488,0.110005,0.001203,0.141732,1.288421,0.000269,1.036967


The table below is sorted by lift. This is a better metric to use while creating association rules, since it tells you how much more a likely customer will buy an item given that he already has the other items in his basket.

For the first two examples, a customer is twice as likely to buy yogurt and milk if he/she has already bought sausages, or sausages if he/she has already bought yogurt and milk.

An interesting association here is between citrus fruit and specialty chocolate (lift of 1.65) and between tropical fruit and flour (lift of 1.61). However, since the dataset is limited (only ~14000 baskets) and the support for the new rules is extremely low, this could be due to chance as well. 

In [28]:
associations.sort_values('lift', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
238,(sausage),"(whole milk, yogurt)",0.060349,0.011161,0.001470,0.024363,2.182917,0.000797,1.013532
235,"(whole milk, yogurt)",(sausage),0.011161,0.060349,0.001470,0.131737,2.182917,0.000797,1.082219
234,"(whole milk, sausage)",(yogurt),0.008955,0.085879,0.001470,0.164179,1.911760,0.000701,1.093681
239,(yogurt),"(whole milk, sausage)",0.085879,0.008955,0.001470,0.017121,1.911760,0.000701,1.008307
87,(citrus fruit),(specialty chocolate),0.053131,0.015973,0.001403,0.026415,1.653762,0.000555,1.010726
86,(specialty chocolate),(citrus fruit),0.015973,0.053131,0.001403,0.087866,1.653762,0.000555,1.038081
236,"(yogurt, sausage)",(whole milk),0.005748,0.157923,0.001470,0.255814,1.619866,0.000563,1.131541
237,(whole milk),"(yogurt, sausage)",0.157923,0.005748,0.001470,0.009310,1.619866,0.000563,1.003596
123,(tropical fruit),(flour),0.067767,0.009757,0.001069,0.015779,1.617141,0.000408,1.006118
122,(flour),(tropical fruit),0.009757,0.067767,0.001069,0.109589,1.617141,0.000408,1.046969
